# Integração de Monte Carlo

## $ \S 1 $ Introdução

Nas regras do trapézio e de Simpson, escolhemos $ N + 1 $ pontos igualmente espaçados no intervalo de integração $ [a, b] $ como membros da "amostra" tomada para os valores da função $ f $ a ser integrada. 

Estes métodos são chamados de **determinísticos** porque o resultado fornecido é completamente determinado pelos parâmetros. Isto é, uma vez fixados a função contínua $ f \colon [a, b] \to \mathbb R $ e o número $ N + 1 $ de pontos, cada aplicação resulta no mesmo valor. Além disto, temos uma estimativa precisa do erro cometido em função de $ N $.

Em contraste, o método da *integração de Monte Carlo* que estudaremos neste caderno é **probabilístico**. A idéia aqui é escolher os pontos onde $ f $ será avaliada *aleatoriamente* dentro do intervalo $ [a, b] $. Desta maneira, cada aplicação fornece uma aproximação diferente, mesmo mantendo-se $ N $ fixo. Ademais, não podemos garantir que o erro envolvido na aproximação seja pequeno para $ N $ grande; podemos apenas provar que a probabilidade de que o erro exceda um certo valor decai conforme $ N $ cresce.

Na verdade a integração de Monte Carlo é um caso especial de um método muito mais geral e extremamente útil, amplamente empregado em Física, Química, Engenharia e outras áreas, o *método de Monte Carlo*.

 ## $ \S 2 $ Descrição do método de Monte Carlo
 
Por **método de Monte Carlo** entende-se uma classe de algoritmos que consistem da execução de um alto número $ N $ de *experimentos* ou *ensaios* e da análise estatística dos resultados obtidos para se obter resultados numéricos. 

**Exemplo 1:** Considere o disco $ D $ de equação
$$
x^2 + y^2 \le 1
$$
no plano $ \mathbb R^2 $, cuja área é $ \pi $ e que está contido no quadrado $ Q $ de área $ 4 $ descrito pelas desigualdades
$$
\lvert{x}\rvert \le 1, \quad \lvert{y}\rvert \le 1.
$$
A probabilidade de que um ponto escolhido de forma aleatoriamente dentro deste quadrado esteja dentro do disco é o quociente entre as áreas, 
$$
\frac{\text{área}(D)}{\text{área}(Q)} = \frac{\pi}{4}.
$$
Suponha que escolhamos aleatoriamente $ 1, 2, 3, \dots $ até um total de $ N $ pontos dentro de $ Q $, e seja $ n $ o número destes pontos pertencentes ao disco $ D $. Um teorema da Teoria da Probabilidade chamado de **Lei dos Grandes Números** diz que o quociente entre o número de "sucessos" $ n $ e o número total $ N $ de pontos na amostra tende à probabilidade do sucesso:
$$
\lim_{N \to \infty} \frac{n}{N} = \frac{\pi}{4}.
$$
Mais ainda, dado $ \varepsilon > 0 $ positivo qualquer,
$$
\lim_{N \to \infty} \text{pr}\bigg(\Big\lvert \frac{n}{N} - \frac{\pi}{4}\Big\rvert \le  \varepsilon \bigg) = 1.
$$
Em palavras, a probabilidade de que a fração entre o número de pontos dentro do disco ($ n $) e o número total de pontos ($ N $) difira de $ \frac{\pi}{4} $ por mais que $ \varepsilon $ tende a zero conforme o número de experimentos aumenta!

## $ \S 3 $ Implementação do método de Monte Carlo

A implementação do método de Monte Carlo genérico é muito simples:

In [21]:
def monte_carlo(ensaio, N):
    """Dada uma função 'ensaio' (a ser implementada separadamente pelo usuário)
    que simula um único experimento e o número N de experimentos a serem
    realizados, contamos o número 'n' de sucessos e retornamos n / N. 
    'ensaio' deve ser uma função sem argumentos que retorna True ou False. """
    n = 0
    for _ in range(N):
        if ensaio():
            n += 1
            
    return n / N

📝 Para simular escolhas aleatórias, iremos utilizar as seguintes funções do módulo `numpy.random`:
* `randint(a, b)` retorna um inteiro aleatório entre os inteiros $ a $ e $ b - 1 $ (inclusive);
* `rand()` retorna um `float` aleatório entre $ 0 $ e $ 1 $.

Em ambos os casos podemos também gerar um array de dimensões especificadas contendo números dos tipos acima.

**Exemplo:**

In [65]:
from numpy.random import rand, randint


print(rand(5))
print(rand(3, 3), end='\n\n')

print(randint(0, 10, size=5))
print(randint(0, 10, size=(2, 4)), end='\n\n')

print(type(randint(0, 10, size=(2, 2))))

[0.58626384 0.3398408  0.7642411  0.05080061 0.82798257]
[[0.79337257 0.94098394 0.17489007]
 [0.52055735 0.42378416 0.33712468]
 [0.27238872 0.7868486  0.7266297 ]]

[2 3 8 4 8]
[[4 7 0 2]
 [4 8 7 1]]

<class 'numpy.ndarray'>


## $ \S 4 $ Sobre o significado da palavra "aleatório"

⚠️ **Observação:** Seja $ \Omega $ um conjunto. A distribuição **uniforme** de probabilidade em $ \Omega $ é aquela que atribiu a cada um de seus elementos $ \omega \in \Omega $ a mesma probabilidade.

No caso em que $ \Omega $ é discreto e finito, digamos com $ N $ elementos, isto significa que
$$
    \text{pr}(\omega) = \frac{1}{N} \qquad \text{para todo $ \omega \in \Omega $}.
$$
No caso contínuo, digamos em que $ \Omega $ é um subconjunto de $ \mathbb R $, isto significa simplesmente que a probabilidade de que um evento pertença a um subconjunto $ S \subset \Omega $ é dada por 
$$
    \frac{\text{comprimento}(S)}{\text{comprimento}(\Omega)} = \frac{\int_S 1\,dx}{\int_\Omega 1\,dx}
$$
Analogamente, com área, volume, etc. em lugar de comprimento, para dimensões mais altas. Nos casos que consideraremos, a integral no denominador será sempre finita.

Sem querer entrar em discussões mais profundas sobre o significado desta palavra, por "escolha **aleatória**" neste caderno deve-se entender simplesmente uma escolha feita de acordo com uma distribuição *uniforme* de probabilidade. Em geral o significado estará claro do contexto, como no exemplo acima.

## $ \S 5 $ Obtendo estimativas para $ \pi $ usando o método de Monte Carlo

**Problema 1:** Use o método de Monte Carlo e o Exemplo 1 para obter uma estimativa para o valor de $ \pi $.

In [19]:
from numpy.random import rand


def pertence_ao_disco() -> bool:
    # 'esticar' transforma o intervalo [0, 1] no intervalo [-1, 1]:
    esticar = lambda x: 2 * x - 1
    
    # Escolha números aleatórios x, y entre -1 e 1.
    # Recorde que rand() escolhe um float aleatório em [0, 1].
    x = esticar(rand())
    y = esticar(rand())
    
    return (x**2 + y**2 <= 1)


In [20]:
from numpy import pi


N = 10**4
for k in range(10):
    print(4 * monte_carlo(pertence_ao_disco, N))

print(pi)

3.1424
3.1404
3.16
3.1096
3.1472
3.134
3.0968
3.1552
3.1412
3.1404
3.141592653589793


**Problema 2:** Um teorema de E. Cèsaro (1850—1906) diz que a probabilidade de que dois naturais escolhidos ao acaso sejam relativamente primos é de $ \frac{6}{\pi^2} $. (Dois inteiros $ m $ e $ n $ são *relativamente primos* caso $ \text{mdc}(m, n) = 1 $.)

(a) Construa um experimento que escolhe dois números aleatórios $ m $ e $ n $ entre $ 1 $ e $ 10^6 $ e retorna `True` caso eles sejam relativamente primos e `False` caso contrário.

(b) Use o método de Monte Carlo e o teorema citado para obter uma aproximação para $ \pi $.

In [52]:
from numpy import sqrt, pi
from numpy.random import randint


def mdc(a, b):
    """Retorna o mdc de dois inteiros a, b. """
    assert isinstance(a, int) and isinstance(b, int)
    if b == 0:
        return a
    else:
        return mdc(b, a % b)

    
def ensaio_rel_primo():
    m = randint(1, 10**5)
    n = randint(1, 10**5)
    return (mdc(m, n) == 1)


N = 10**4
resultado = monte_carlo(ensaio_rel_primo, N)
print(sqrt(6 / resultado))
print(pi)

3.1510067105223283
3.141592653589793


## $ \S 6 $ Aplicação a integrais de funções de uma variável

Recorde do curso de Cálculo 1 que a integral
$$
\int_a^b f(x)\,dx
$$
é aproximada por *somas de Riemann*
$$
\sum_{i=1}^N f(\xi_i) \Delta_i
$$
onde
$$
[x_0, x_1],\ [x_1, x_2],\ \dots,\ [x_{N-1}, x_{N}]
$$
formam uma partição de $ [a, b] $,
$$
\Delta_i = x_i - x_{i-1}
$$
e os pontos
$$
\xi_i \in [x_{i-1}, x_i] \qquad (i = 1, \dots, N)
$$
podem ser escolhidos arbitrariamente. De fato, a integral é *definida* como o limite de somas deste tipo conforme o maior dos comprimentos $ \Delta_i $ vai a zero.

Considere o caso especial em que $ \Delta_i = \frac{b-a}{N} $ para cada $ i $, ou seja, os pontos são
$$
x_i = a +  i\frac{b-a}{N}
$$
e estão igualmente espaçados. Então a soma de Riemann se reduz a 
$$
\frac{b-a}{N}\sum_{i=1}^N f(\xi_i) = (b - a)\bigg(\frac{1}{N}\sum_{i=1}f(\xi_i)\bigg).
$$    
O termo entre parênteses aqui pode ser visto como a média de uma amostra de $ N $ valores de $ f $ no intervalo, sujeitos à condição de que o $ i $-ésimo ponto pertence ao $ i $-ésimo intervalo da partição.

Suponha agora que os pontos $ \xi_i $ ($ i = 1, \dots, N $) sejam escolhidos *aleatoriamente* dentre de $ [a, b] $, sem qualquer restrição (ou seja, não imporemos que $ \xi_i \in [x_{i-1}, x_i] $). A Lei dos Grandes Números ainda garante que
$$
(b - a)\bigg(\frac{1}{N}\sum_{i=1}^Nf(\xi_i)\bigg) \to \int_a^b f(x)\,dx \qquad \text{conforme }N \to \infty.
$$
A **integração de Monte Carlo** consiste da aproximação da integral à direita pelo valor à esquerda.

## $ \S 6 $ Implementação da integração de Monte Carlo em 1 dimensão

In [23]:
def integracao_monte_carlo(f, a, b, N):
    from numpy.random import rand
    
    # Transforma o intervalo [0, 1] no intervalo [a, b]:
    esticar = lambda x: a + (b - a) * x
    
    soma = 0
    for _ in range(N):
        x = esticar(rand())
        soma += f(x)
    
    return (b - a) * soma / N

In [24]:
from numpy import pi


f = lambda x: 1 / (1 + x**2)
a = 0
b = 1
N = 10000

print(pi)
print(4 * integracao_monte_carlo(f, a, b, N))

3.141592653589793
3.1456573534599523


## $ \S 7 $ Problema da agulha

Identifique o $ \mathbb R^2 $ com um pedaço de chão e suponha que as retas de equação
$$
x = 0\ , \quad x = \pm 1\ , \quad x = \pm 2\ , \dots, \quad x = \pm n\ , \quad \dots \qquad (n \in \mathbb N)
$$
estejam todas demarcadas. Jogamos uma agulha de comprimento $ c $ no chão de modo que sua posição e direção ao atingir um estado de repouso sejam determinadas aleatoriamente.

(a) Calcule a *probabilidade* de que a agulha cruze uma linha demarcada. Sua resposta deve depender apenas de $ c $.

*Dica:* Para facilitar as considerações, seja $ P = (x_0, y_0) $ a posição de um das pontas da agulha (escolhida uma vez por todas, antes do lançamento) e seja $ \theta \in [0, 2\pi) $ o ângulo que o vetor que liga esta ponta com a outra forma com o eixo-$x$ positivo. Podemos supor sem perda de generalidade que $ 0 \le x_0 \le 1 $. Note que o valor de $ y $ é irrelevante para se determinar se a agulha cruza uma linha demarcada. Determine o conjunto $ R $ de todos os pares
$$ (\theta, x(\theta)) $$
para os quais uma agulha de comprimento $ c $ com ponta inicial $ (x(\theta), y) $ formando um ângulo de $ \theta $ com o eixo-$ x $ positivo não cruza uma linha demarcada. A probabilidade será dada por
$$
\frac{\text{área}(R)}{2\pi}.
$$

(b) Use o método de Monte Carlo e o seu resultado do item (a) para estimar $ \pi $.


## $ \S 8 $ Integração de Monte Carlo em dimensão $ \ge 2 $

Para integração de funções de apenas uma variável, em geral os métodos determinísticos devem ser preferidos em relação à integração de Monte Carlo, por fornecerem resultados bem mais precisos com um número menor de computação.

Contudo, em dimensões mais altas estes métodos não funcionam tão bem. A primeira dificuldade encontrada é que o número de operações necessárias aumenta rapidamente com a dimensão. Grosso modo, se para garantir uma determinada precisão eram necessárias $ N $ avaliações em dimensão $ 1 $, em dimensão $ 3 $ serão necessárias $ N^3 $. Outra dificuldade é que, enquanto em dimensão $ 1 $ quase sempre integramos sobre um intervalo, em dimensões $ \ge 2 $ a fronteira da região onde será calculada a integral pode ser muito mais irregular e complicada.

📝 O método de Monte Carlo é especialmente conveniente para o cálculo de áreas e volumes de regiões definidas através de desigualdades.

In [34]:
def int_monte_carlo_multi_dim(f, limites, N):
    from numpy.random import rand
    from math import prod
    
    def esticar(a, b, x):
         return a + (b - a) * x
    
    assert isinstance(N, int)
    assert len(limites) % 2 == 0
    
    dimensao = len(limites) // 2
    
    soma = 0
    for _ in range(N):
        x = [0 for k in range(dimensao)]
        for k in range(dimensao):
            x[k] = esticar(rand(), limites[2 * k], limites[2 * k + 1])
        soma += f(*x)
    
    volume_total = prod(limites[2 * k + 1] - limites[2 * k] for k in range(dimensao))
    return volume_total * soma / N

**Problema 3:** Calcule o volume $ V $ do sólido em $ \mathbb R^3 $ determinado pelas desigualdades
$$
0 \le x \le y \le z \le 1
$$
com um erro menor que $ 10^{-3} $.

In [35]:
def ensaio_P3():
    
    x = rand()
    y = rand()
    z = rand()
    
    return (x <= y) and (y <= z)
    

N = 10**5
print(monte_carlo(ensaio_P3, N))
print()

0.1677



**Problema 4:** Estime a integral dupla
$$
    \int_{-1}^{2}\int_{-1}^2 xy\big(2-x^2\big)\big(2-xy\big)\,dx\,dy
$$
usando integração de Monte Carlo.

In [63]:
def f(x, y):
    return x * y * (2 - x**2) * (2 - x * y)


limites = [-1, 2, -1, 2]
N = 10**5
print(int_monte_carlo_multi_dim(f, limites, N))

1177.0498756611084
